# 6장. 순환신경망
#### 주석에 해당하는 코드를 적어주세요.

In [1]:
!pip install torch torchvision

1. 필요한 라이브러리 불러오기

In [12]:
import torch

# 신경망 모델들 포함
import torch.nn as nn
# 경사하강법 알고리즘
import torch.optim as optim
# 텐서에 초기값을 주기 위해 필요한 함수들
import torch.nn.init as init

import numpy as np

2. 하이퍼파라미터 설정

In [3]:
n_hidden = 35 
lr = 0.01
epochs = 1000

string = "hello pytorch. how long can a rnn cell remember? show me your limit!"
chars =  "abcdefghijklmnopqrstuvwxyz ?!.,:;01"

In [4]:
# 문자들을 리스트로 바꾸어 char_list로 저장한다. 
char_list = [i for i in chars]
# char_list의 길이(=문자의 개수)를 저장한다.
n_letters = len(char_list)

3. 함수 생성
- 문장 input시 연산 가능한 원-핫 벡터로 바꾸는 함수 : string_to_onehot
- One-hot 벡터를 다시 문자로 바꿔주는 함수 : onehot_to_word

In [20]:
def string_to_onehot(string):
    # 시작 토큰 만들기
    start = np.zeros(shape=len(char_list), dtype=int)
    # 끝 토큰 만들기
    end = np.zeros(shape=len(char_list), dtype=int)

    start[-2] = 1
    end[-1] = 1

    for i in string:
        # 문자가 몇번째 문자인지를 idx에 저장한다.(a:0, b:1, c:2,...)
        idx = char_list.index(i)
        # 0으로만 구성된 배열을 만들어준다. ([0 0 0 … 0 0])
        zero = np.zeros(shape=n_letters, dtype=int)
        # 해당 문자 인덱스만 1로 바꿔준다. (b: [0 1 0 … 0 0])
        zero[idx]=1
        # start와 새로 생긴 zero를 붙이고 이를 start에 할당한다.
        start = np.vstack([start, zero])
    # 문자열이 다 끝나면 쌓아온 start와 end를 붙여준다.
    output = np.vstack([start, end])
    return output

In [21]:
def onehot_to_word(onehot_1):
    # 텐서를 입력값으로 받아, 넘파이 배열로 바꿔준다.
    onehot = torch.Tensor.numpy(onehot_1)
    # onehot이 1인 지점을 인덱스로 잡아 char_list에서 뽑아낸다.
    return char_list[onehot.argmax()]

4. 순환 신경망 클래스

RNN with 1 hidden layer


In [40]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        # self.i2o = nn.Linear(input_size + hidden_size, output_size)
        # self.act_fn = nn.Tanh()

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(hidden_size, output_size)
        self.act_fn = nn.Tanh()

    
    def forward(self, input, hidden):
        # 입력과 hidden state를 cat함수로 붙인다.
        combined = torch.cat((input, hidden), dim=1)
        # 붙인 값을 i2h에 통과시켜 hidden state를 업데이트한다.
        hidden = self.act_fn(self.i2h(combined))
        # 붙인 값을 i2o에 통과시켜 output을 계산한다.
        output = self.i2o(hidden)
        return output, hidden
    
    # hidden state 초기화 함수 정의 
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)
    
rnn = RNN(n_letters, n_hidden, n_letters)

In [41]:
# 손실함수 설정
loss_func = nn.MSELoss()
# 최적화함수 설정
optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)

5. Train 학습 부분

In [42]:
# 우리가 학습하고자 했던 문장(sting)을 원-핫 벡터로 변환한 넘파이 배열(sting_to_onehot)을 다시 토치 텐서 형태로 바꿔준다.
one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor())

for i in range(epochs):
    optimizer.zero_grad()
    # hidden state 초기화
    hidden = rnn.init_hidden()
    
    # 문자열 전체에 대한 손실을 구하기 위한 변수 지정
    total_loss = 0
    for j in range(one_hot.size()[0]-1):
        # 앞의 글자 입력시 : input 지정
        input_ = one_hot[j:j+1, :]
        # 뒤의 글자 구하기 : target 지정 
        target = one_hot[j+1]
        output, hidden = rnn.forward(input_, hidden)
        
        loss = loss_func(output.view(-1),target.view(-1))
        total_loss += loss

    total_loss.backward()
    optimizer.step()

    if i % 10 == 0:
        print(total_loss)

tensor(3.3568, grad_fn=<AddBackward0>)
tensor(1.5132, grad_fn=<AddBackward0>)
tensor(1.1180, grad_fn=<AddBackward0>)
tensor(0.8316, grad_fn=<AddBackward0>)
tensor(0.6839, grad_fn=<AddBackward0>)
tensor(0.5334, grad_fn=<AddBackward0>)
tensor(0.4203, grad_fn=<AddBackward0>)
tensor(0.3483, grad_fn=<AddBackward0>)
tensor(0.2895, grad_fn=<AddBackward0>)
tensor(0.2608, grad_fn=<AddBackward0>)
tensor(0.2412, grad_fn=<AddBackward0>)
tensor(0.2303, grad_fn=<AddBackward0>)
tensor(0.2003, grad_fn=<AddBackward0>)
tensor(0.1844, grad_fn=<AddBackward0>)
tensor(0.1951, grad_fn=<AddBackward0>)
tensor(0.1661, grad_fn=<AddBackward0>)
tensor(0.1550, grad_fn=<AddBackward0>)
tensor(0.1639, grad_fn=<AddBackward0>)
tensor(0.1472, grad_fn=<AddBackward0>)
tensor(0.1394, grad_fn=<AddBackward0>)
tensor(0.1324, grad_fn=<AddBackward0>)
tensor(0.1404, grad_fn=<AddBackward0>)
tensor(0.1252, grad_fn=<AddBackward0>)
tensor(0.1197, grad_fn=<AddBackward0>)
tensor(0.1317, grad_fn=<AddBackward0>)
tensor(0.1201, grad_fn=<A

6. Test

In [43]:
start = torch.zeros(1,n_letters)
start[:,-2] = 1

with torch.no_grad():
    hidden = rnn.init_hidden()
    # 처음 입력 지정 : start token
    input_ = start
    # output string에 문자들을 계속 붙여준다
    output_string = ""

    # 원래는 end token이 나올때 까지 반복하는게 맞으나 끝나지 않아서 string의 길이로 정했습니다.
    for i in range(len(string)):
        output, hidden = rnn.forward(input_, hidden)
        # 결과값을 문자로 바꿔서(onehot_to_word) output_string에 붙여준다.
        output_string += onehot_to_word(output.data)
        # 이번의 결과값이 다음의 입력값이 되도록 지정한다.
        input_ = output

print(output_string)

hellrni hyw hchmr howpych.whme le moehythrchhw meehmehpehow ho hoehm
